# Vector Map

The pygrass interface for raster maps is divided in 2 classes that represent different ways to interact with vector.

The [**Vector**](http://grass.osgeo.org/grass71/manuals/libpython/pygrass.vector.html#pygrass.vector.Vector) class it is used to work with vector data *without topology*

The [**VectorTopo**](http://grass.osgeo.org/grass71/manuals/libpython/pygrass.vector.html#pygrass.vector.VectorTopo) class instead it is used to work with vector data *with topology*; **this is the class that you should use in most of the case**

We can create a vector map with two approaches:

1) write the vector in two steps:

    a. write the geometry features of the vector map
    b. write the values in the attribute table

2) write the vector map, geometry features and tabel attrinute in one step

## Write a vector map in two steps

### Write the geometry features

In [ ]:
from grass.pygrass.vector import VectorTopo
from grass.pygrass.vector.geometry import Point

In [ ]:
new = VectorTopo('newvect')

In [ ]:
new.open('w')

Define some geometry features that will be add to the vector map

In [ ]:
point0 = Point(636981.336043, 216517.602235)
point1 = Point(637209.083058, 217970.129540)

Write the geometry to the vector map

In [ ]:
new.write(point0)
new.write(point1)

In [ ]:
new.close()

### Read the geometry features of the vector map

In [ ]:
new.open('r')

In [ ]:
new[1]

In [ ]:
new[2]

In [ ]:
new[1].attrs  # No table attribute has been created yet

In [ ]:
new.build()

### Write the attribute values

Create a connection with the database

In [ ]:
from grass.pygrass.vector.table import Link

# set the layer name
new.layer = 1
# create a Link object
link = Link(layer=new.layer, name=new.name, table=new.name, key='cat',
            database='$GISDBASE/$LOCATION_NAME/$MAPSET/sqlite/sqlite.db',
            driver='sqlite')

Open the vector map in read and write mode to not loose the geometry features of the vector map

In [ ]:
new.open('rw')

Add the link to dblinks

In [ ]:
new.dblinks.add(link)

Define the columns of the attribute table

In [ ]:
cols = [(u'cat',       'INTEGER PRIMARY KEY'),
        (u'name',      'VARCHAR')]

Retrive the Table object and create the table in the database

In [ ]:
new.table = new.dblinks[0].table()

# create the table in the DB
# new.table.columns.create(cols)
new.table.create(cols)

insert the values in the table

In [ ]:
new.table.insert([(1, 'pub'), (2, 'resturant')], many=True)

Save the changes

In [ ]:
new.table.conn.commit()
new.close()

## Write a new vector map in one step

Remove the previous map

In [ ]:
new.remove()

Open the map and create the database link and table, with:

In [ ]:
new = VectorTopo('newvect')
new.open('w', tab_cols=cols)

Write the geometry feature and the attribute

In [ ]:
new.write(point0, ('pub', ))
new.write(point1, ('restaurant', ))

save the changes and close the map

In [ ]:
new.table.conn.commit()
new.close()

In [ ]:
new.open('r')
new[1].attrs['name']

In [ ]:
new.close()

### Read the new vector map

In [ ]:
new = VectorTopo('newvect')
new.open('r')

Read the keys of attribute table for the first feature

In [ ]:
first=new[1]
first.read()

Read the values of attribute table for the first feature

In [ ]:
first.attrs

In [ ]:
first.attrs['name']

Replace value of *name* columns for the first feature

In [ ]:
new[1].attrs['name'] = 'pizza'

In [ ]:
new[1].attrs['name']

In [ ]:
new[1].attrs['cat']

In [ ]:
new.cat(1, 'points')

In [ ]:
pnt = new.cat(1, 'points')[0]

In [ ]:
pnt.cat

In [ ]:
pnt.attrs['name']

In [ ]:
new.close()

## Rewrite

Open the map in read-write mode

In [ ]:
new.open('rw')

Create a new geometry feature:

In [ ]:
point010 = Point(point0.x + 10, point0.y + 10)

Check the existing values:

In [ ]:
print(new[1] == point010)
print(new[1].attrs.values())

Rewrite the values:

In [ ]:
new.rewrite(point010, cat=1, attrs=('Irish Pub',))

And commit the changes in the database and close the vector map

In [ ]:
new.table.conn.commit()
new.close()

Check the values in the vector map

In [ ]:
new.open('r')
print(new[1] == point010)
print(new[1].attrs.values())
new.close()

## Vector class methods

Check if a map exists with **exist** method.

In [ ]:
new.exist()

Check if a map is open or not, with **is_open** method.

In [ ]:
new.is_open()

Let's open and ask for **bbox()**

In [ ]:
new.open('r')

In [ ]:
new.bbox()

Iterate between each line of the vector map

In [ ]:
for pnt in new:
    print(pnt, pnt.attrs.values())

Get a dictionary with the number of primitive that are present in the vector map

In [ ]:
new.num_primitives()

Get a number of primitive of the vector map for a selected geometry feature. 

In [ ]:
new.num_primitive_of('point')

In [ ]:
new.person

**viter()** is an iteration method that can be use to perform tasks on each geometry primitive.

In [ ]:
count = 0
for pnt in new.viter("lines"):
    count += 1
print("Lines {}".format(count))
count = 0
for pnt in new.viter("points"):
    count += 1
print("Points {}".format(count))

# Object contained in a Vector map

## DBLinks

A vector map may have differents attributes tables connected using different database link. The **dblinks** attribute containes all this information.

More info about <a href="http://grass.osgeo.org/grass71/manuals/libpython/pygrass.vector.html#pygrass.vector.table.DBlinks" target="_blank"><b>DBLinks class</b></a>

In [ ]:
new.dblinks

The **num_dblinks** method return the number of link object that are available for the vector map

In [ ]:
new.dblinks.num_dblinks()

Then we can choose the database connection using the **index**, the **layer** or the connection **name**:

In [ ]:
new.dblinks.by_index(0)

In [ ]:
new.dblinks.by_layer(1)

In [ ]:
new.dblinks.by_name('newvect')

We can add new connection to the vector map with the **add** method:

In [ ]:
new.dblinks.add

and remove the connection with the **remove** method

In [ ]:
new.dblinks.remove

## Link

The *Link* class it is used for the link between vector map and attribute table.

More info about <a href="http://grass.osgeo.org/grass71/manuals/libpython/pygrass.vector.html#pygrass.vector.table.Link" target="_blank"><b>Link class</b></a>

In [ ]:
link = new.dblinks[0]

You can obtain more information about the connection link

In [ ]:
link.database

In [ ]:
link.name

In [ ]:
link.table_name

In [ ]:
link.driver

In [ ]:
link.key

In [ ]:
link.layer

The **connection** method return a Connection object that follow the [PEP-249](http://www.python.org/dev/peps/pep-0249/)

In [ ]:
link.connection()

therefore we can use directly the connection with the db, with:

In [ ]:
conn = link.connection()
cur = conn.cursor()
cur.execute("SELECT * FROM {nv}".format(nv=new.name))
cur.fetchall()

The **table** method return a **Table** object that will be show in the following part.

In [ ]:
link.table()

## Table

The **Table** object describe the attribute table, and it is possible to *do query on attributes*.

More info about <a href="http://grass.osgeo.org/grass71/manuals/libpython/pygrass.vector.html#pygrass.vector.table.Table" target="_blank"><b>Table class</b></a>

In [ ]:
new.table.columns

Obtain all the data from the table

In [ ]:
cur = new.table.execute()
cur.fetchall()

or filter the select

In [ ]:
new.table.filters.select('name').where('cat=1')

In [ ]:
new.table.filters.get_sql()

In [ ]:
cur = new.table.execute()
cur.fetchone()[0]

In [ ]:
new.close()

# Summary

We saw how to:

* write a vector map with only the geometry features;
* add a new database connection, create and fill the attributes table;
* read, iterate and modify an existing vector map;
* select some values from an attribute table, change, add, cast and remove columns;

# Exercise

Below, we define a function to create random point

In [ ]:
# import the random function
import random

# import the Point and Region functions?
from grass.pygrass.vector.geometry import Point
from grass.pygrass.gis.region import Region

# define a function to produce random points
def get_random_points(num):
    # inside current GRASS' region of course
    reg = Region()
    # loop over a series of numbers from 0 up to...
    for _ in xrange(0, num):
        # use the function randrange() to get both x and y random numbers
        x = random.randrange(reg.south, reg.north)
        y = random.randrange(reg.west, reg.east)
        # at the end of each loop we need to explicitly catch the pairs of x and y numbers
        # in order to produce a point
        yield Point(x, y)


Using the **get_random_points** function write a function that return a new vector points map

In [ ]:
# Here write the function

In [ ]:
rand_pnts = rand_vect_points('random_points', npoints=10)

In [ ]:
# Here create the new vector

use the **viter()** function, just to prove the iteration concept 